In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import os
import psycopg2 as pg
import pandas.io.sql as pd_sql
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

/home/zhouza/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
df_virus_load = pd.read_csv('Data/train.csv',sep=',')
df_spray_load = pd.read_csv('Data/spray.csv',sep=',')
df_weather_load = pd.read_csv('Data/weather.csv',sep=',')

In [40]:
df_virus_load.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
Date                      10506 non-null object
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null object
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 985.0+ KB


In [41]:
df_spray_load.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14835 entries, 0 to 14834
Data columns (total 4 columns):
Date         14835 non-null object
Time         14251 non-null object
Latitude     14835 non-null float64
Longitude    14835 non-null float64
dtypes: float64(2), object(2)
memory usage: 463.7+ KB


In [42]:
df_weather_load.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2944 entries, 0 to 2943
Data columns (total 22 columns):
Station        2944 non-null int64
Date           2944 non-null object
Tmax           2944 non-null int64
Tmin           2944 non-null int64
Tavg           2944 non-null object
Depart         2944 non-null object
DewPoint       2944 non-null int64
WetBulb        2944 non-null object
Heat           2944 non-null object
Cool           2944 non-null object
Sunrise        2944 non-null object
Sunset         2944 non-null object
CodeSum        2944 non-null object
Depth          2944 non-null object
Water1         2944 non-null object
SnowFall       2944 non-null object
PrecipTotal    2944 non-null object
StnPressure    2944 non-null object
SeaLevel       2944 non-null object
ResultSpeed    2944 non-null float64
ResultDir      2944 non-null int64
AvgSpeed       2944 non-null object
dtypes: float64(1), int64(5), object(16)
memory usage: 506.1+ KB


zhouza@Lilith:/media/zhouza/Storage/Metis/MetisProject03/Data\$ ls

spray.csv  train.csv  weather.csv  west_nile

zhouza@Lilith:/media/zhouza/Storage/Metis/MetisProject03/Data\$ scp *.csv my_aws:~

spray.csv                                     100%  778KB 777.5KB/s   00:00    
train.csv                                     100% 1755KB   1.7MB/s   00:00    
weather.csv                                   100%  350KB 349.8KB/s   00:00 


ubuntu@ip-172-31-25-151:~\$ psql

psql (9.5.12)

Type "help" for help.

ubuntu=# CREATE DATABASE west_nile;

CREATE DATABASE

ubuntu=# \connect west_nile

You are now connected to database "west_nile" as user "ubuntu".

In [2]:
# Postgres info to connect

connection_args = {
    'host': '18.222.80.148', # You'll have to update this to your IP
    'user': 'ubuntu',    # username
    'dbname': 'west_nile',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

connection = pg.connect(**connection_args)
cursor = connection.cursor()

In [341]:
cursor.execute('BEGIN;')
query = '''
    CREATE TABLE spray_load
    (date TEXT, time TEXT, latitude DOUBLE PRECISION, longitude DOUBLE PRECISION)
    ;
    COPY spray_load FROM '/home/ubuntu/spray.csv' DELIMITER ',' CSV HEADER;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [342]:
cursor.execute('BEGIN;')
query = '''
    CREATE TABLE virus_load
    (date TEXT, address TEXT, species TEXT, block INT, street TEXT, trap TEXT, address_short TEXT, 
    latitude DOUBLE PRECISION, longitude DOUBLE PRECISION, address_accuracy INT, freq INT, virus_present INT
    )
    ;
    COPY virus_load FROM '/home/ubuntu/train.csv' DELIMITER ',' CSV HEADER;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [343]:
cursor.execute('BEGIN;')
query = '''
    CREATE TABLE weather_load
    (station INT,
    date TEXT,
    tmax INT,
    tmin INT,
    tavg TEXT,
    delta TEXT,
    dewpt INT,
    wetbulb TEXT,
    heat TEXT,
    cool TEXT,
    sunrise TEXT,
    sunset TEXT,
    codesum TEXT,
    snowdepth TEXT,
    snowwater TEXT,
    snowfall TEXT,
    precip TEXT,
    pressure TEXT,
    sealvlpressure TEXT,
    windspeed DOUBLE PRECISION,
    winddirection INT,
    avgspeed TEXT 
    );
    COPY weather_load FROM '/home/ubuntu/weather.csv' DELIMITER ',' CSV HEADER;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [344]:
# convert date to date type instead of string
cursor.execute('BEGIN;')
query = '''
    ALTER TABLE spray_load
    ADD COLUMN date_type date
    ;
    ALTER TABLE virus_load
    ADD COLUMN date_type date
    ;
    ALTER TABLE weather_load
    ADD COLUMN date_type date
    ;
    UPDATE spray_load
    SET date_type = to_date(date,'YYYY-MM-DD')
    ;
    UPDATE virus_load
    SET date_type = to_date(date,'YYYY-MM-DD')
    ;
    UPDATE weather_load
    SET date_type = to_date(date,'YYYY-MM-DD')
    ;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [345]:
cursor.execute('BEGIN;')
query = '''
    CREATE TABLE spray
    AS SELECT DISTINCT * FROM spray_load
    ;
    CREATE TABLE weather
    AS SELECT DISTINCT * FROM weather_load
    ;
    CREATE TABLE virus
    AS SELECT DISTINCT * FROM virus_load
    ;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [347]:
def find_data_inconsistency(table_name):
    '''For a table in a database, identify distinct non-numeric values and their counts
    '''
    df = pd_sql.read_sql('SELECT * FROM ' + table_name + ';', connection)
    column_list = df.dtypes[(df.dtypes=='object') 
                                         & (df.columns != 'date') 
                                         & (df.columns != 'date_type')].index
    query_list = []
    for i in column_list:
        query_block = '''SELECT \'''' + i + '\' AS colname,' + i + ''' AS val,COUNT(*)\
            FROM ''' + table_name + '''\
            WHERE trim(''' + i + ''') !~ \'[0-9]\'\
            OR trim(''' + i + ''') = \'\'\
            GROUP BY ''' + i
        query_list.append(query_block)
    query = ' UNION '.join(query_list)
    return pd_sql.read_sql(query + ' ORDER BY 1,3 DESC', connection)

In [348]:
def update_table(table_name,column_name,find_value,replace_value):
    '''For a table in a database, replace values in a column
    '''
    query = '''\
        UPDATE ''' + table_name + '''\
        SET ''' + column_name + ''' = ''' + replace_value + '''\
        WHERE TRIM(''' + column_name + ''') = ''' + find_value + ';'
    cursor.execute(query)

In [349]:
# M is frequently used to mean "missing" but should be null
# T is used to mean 0 (per documentation on data)
# - is used when sunrise/sunset data not provided - these should be null

find_data_inconsistency('weather')

,colname,val,count
0,avgspeed,M,3
1,codesum,,1609
2,codesum,RA,296
3,codesum,RA BR,238
4,codesum,BR,110
5,codesum,TSRA RA BR,92
6,codesum,BR HZ,81
7,codesum,RA DZ BR,65
8,codesum,TSRA RA,43
9,codesum,HZ,39


In [350]:
# blanks in codesum aren't actually nulls, they are spaces 
pd_sql.read_sql('SELECT COUNT(*) FROM weather WHERE codesum = \' \';', connection)

,count
0,1609


In [351]:
# correct values found above

cursor.execute('BEGIN;')
for i in ['avgspeed','cool','delta','heat','precip','pressure','sealvlpressure','snowdepth','snowfall','snowwater','tavg','wetbulb']:
    update_table('weather',i,'\'M\'','null')
    
for i in ['precip','snowfall']:
    update_table('weather',i,'\'T\'','\'0\'')
    
for i in ['sunrise','sunset']:
    update_table('weather',i,'\'-\'','null')
    
update_table('weather','codesum','\'\'','\'NONE\'')

In [352]:
# validate update
find_data_inconsistency('weather')

,colname,val,count
0,codesum,NONE,1609
1,codesum,RA,296
2,codesum,RA BR,238
3,codesum,BR,110
4,codesum,TSRA RA BR,92
5,codesum,BR HZ,81
6,codesum,RA DZ BR,65
7,codesum,TSRA RA,43
8,codesum,HZ,39
9,codesum,RA BR HZ,38


In [353]:
cursor.execute('COMMIT;')

In [354]:
# no issues here
find_data_inconsistency('virus')

,colname,val,count
0,address,"South Doty Avenue, Chicago, IL, USA",212
1,address,"South Stony Island Avenue, Chicago, IL, USA",181
2,address,"South Torrence Avenue, Chicago, IL, USA",127
3,address,"South Cottage Grove Avenue, Chicago, IL, USA",112
4,address,"South Vincennes Avenue, Chicago, IL, USA",91
5,address,"Chicago, IL, USA",89
6,address,"West Garfield Boulevard, Chicago, IL, USA",77
7,address,"South Wallace Street, Chicago, IL, USA",8
8,species,CULEX PIPIENS/RESTUANS,4469
9,species,CULEX RESTUANS,2672


In [355]:
# "NA" should be null
find_data_inconsistency('spray')

,colname,val,count
0,time,NA,584


In [356]:
# correct values found above
cursor.execute('BEGIN;')

update_table('spray','time','\'NA\'','null')

In [357]:
# validate update
find_data_inconsistency('spray')

,colname,val,count


In [358]:
cursor.execute('COMMIT;')

In [359]:
# there are 2 weather stations with some different data - look at counts to determine which to make the primary source of data and use the other to supplement
pd_sql.read_sql('SELECT * FROM weather WHERE station = 1', connection).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 23 columns):
station           1472 non-null int64
date              1472 non-null object
tmax              1472 non-null int64
tmin              1472 non-null int64
tavg              1472 non-null object
delta             1472 non-null object
dewpt             1472 non-null int64
wetbulb           1469 non-null object
heat              1472 non-null object
cool              1472 non-null object
sunrise           1472 non-null object
sunset            1472 non-null object
codesum           1472 non-null object
snowdepth         1472 non-null object
snowwater         0 non-null object
snowfall          1472 non-null object
precip            1472 non-null object
pressure          1470 non-null object
sealvlpressure    1467 non-null object
windspeed         1472 non-null float64
winddirection     1472 non-null int64
avgspeed          1472 non-null object
date_type         1472 non-null object
dty

In [360]:
pd_sql.read_sql('SELECT * FROM weather WHERE station = 2', connection).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 23 columns):
station           1472 non-null int64
date              1472 non-null object
tmax              1472 non-null int64
tmin              1472 non-null int64
tavg              1461 non-null object
delta             0 non-null object
dewpt             1472 non-null int64
wetbulb           1471 non-null object
heat              1461 non-null object
cool              1461 non-null object
sunrise           0 non-null object
sunset            0 non-null object
codesum           1472 non-null object
snowdepth         0 non-null object
snowwater         0 non-null object
snowfall          0 non-null object
precip            1470 non-null object
pressure          1470 non-null object
sealvlpressure    1468 non-null object
windspeed         1472 non-null float64
winddirection     1472 non-null int64
avgspeed          1469 non-null object
date_type         1472 non-null object
dtypes: float64(1)

In [361]:
# join data from station 2 to station 1 and create a view with this information

cursor.execute('BEGIN;')

query = '''
    CREATE OR REPLACE VIEW weather_condensed AS
    SELECT t1.date_type date, t1.tmax, t1.tmin, CAST(t1.tavg AS DOUBLE PRECISION),
    CAST(COALESCE(t1.delta, t2.delta) AS DOUBLE PRECISION) delta,
    CAST(COALESCE(t1.dewpt, t2.dewpt) AS DOUBLE PRECISION) dewpt,
    CAST(COALESCE(t1.wetbulb, t2.wetbulb) AS DOUBLE PRECISION) wetbulb,
    CAST(COALESCE(t1.heat, t2.heat) AS DOUBLE PRECISION) heat,
    CAST(COALESCE(t1.cool, t2.cool) AS DOUBLE PRECISION) cool,
    COALESCE(t1.sunrise, t2.sunrise) sunrise,
    COALESCE(t1.sunset, t2.sunset) sunset,
    COALESCE(t1.codesum, t2.codesum) codesum,
    CAST(COALESCE(t1.snowdepth, t2.snowdepth) AS DOUBLE PRECISION) snowdepth,
    CAST(COALESCE(t1.snowfall, t2.snowfall) AS DOUBLE PRECISION) snowfall,
    CAST(COALESCE(t1.precip, t2.precip) AS DOUBLE PRECISION) precip,
    CAST(COALESCE(t1.pressure, t2.pressure) AS DOUBLE PRECISION) pressure,
    CAST(COALESCE(t1.sealvlpressure, t2.sealvlpressure) AS DOUBLE PRECISION) sealvlpressure,
    CAST(COALESCE(t1.windspeed, t2.windspeed) AS DOUBLE PRECISION) windspeed,
    CAST(COALESCE(t1.winddirection, t2.winddirection) AS DOUBLE PRECISION) winddirection,
    CAST(COALESCE(t1.avgspeed, t2.avgspeed) AS DOUBLE PRECISION) avgspeed    
    FROM weather t1 LEFT JOIN weather t2
    ON t1.date_type = t2.date_type
    WHERE t1.station = 1 and t2.station = 2
    ORDER BY 1
    ;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [362]:
pd_sql.read_sql('SELECT * FROM weather_condensed;', connection).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472 entries, 0 to 1471
Data columns (total 20 columns):
date              1472 non-null object
tmax              1472 non-null int64
tmin              1472 non-null int64
tavg              1472 non-null float64
delta             1472 non-null float64
dewpt             1472 non-null float64
wetbulb           1472 non-null float64
heat              1472 non-null float64
cool              1472 non-null float64
sunrise           1472 non-null object
sunset            1472 non-null object
codesum           1472 non-null object
snowdepth         1472 non-null float64
snowfall          1472 non-null float64
precip            1472 non-null float64
pressure          1471 non-null float64
sealvlpressure    1472 non-null float64
windspeed         1472 non-null float64
winddirection     1472 non-null float64
avgspeed          1472 non-null float64
dtypes: float64(14), int64(2), object(4)
memory usage: 230.1+ KB


In [363]:
# confirm there are no duplicates from the weather station join
pd_sql.read_sql('SELECT date,count(*) FROM weather_condensed GROUP BY date HAVING count(*)>1;', connection)

,date,count


In [373]:
# confirm there are no duplicates from the mosquito trap data
pd_sql.read_sql('SELECT date,latitude,longitude,count(*) FROM spray GROUP BY date,latitude,longitude HAVING count(*)>1;', connection)

,date,latitude,longitude,count


In [376]:
# "duplicates" by unique key will need to be summed
pd_sql.read_sql('SELECT date,block,trap,species,latitude,longitude,virus_present,count(*) FROM virus GROUP BY date,block,trap,species,latitude,longitude,virus_present HAVING COUNT(*)>1 ORDER BY 8 DESC;', connection)

,date,block,trap,species,latitude,longitude,virus_present,count
0,2013-06-14,10,T900,CULEX RESTUANS,41.974689,-87.890615,0,10
1,2013-07-25,10,T900,CULEX RESTUANS,41.974689,-87.890615,0,9
2,2013-07-08,10,T900,CULEX RESTUANS,41.974689,-87.890615,0,9
3,2013-08-01,10,T900,CULEX PIPIENS/RESTUANS,41.974689,-87.890615,0,9
4,2007-08-01,22,T086,CULEX PIPIENS,41.688324,-87.676709,0,8
5,2011-06-10,10,T900,CULEX RESTUANS,41.974689,-87.890615,0,8
6,2011-06-17,10,T900,CULEX RESTUANS,41.974689,-87.890615,0,8
7,2009-08-13,10,T900,CULEX PIPIENS/RESTUANS,41.974689,-87.890615,0,7
8,2013-06-21,10,T900,CULEX RESTUANS,41.974689,-87.890615,0,7
9,2013-07-25,10,T900,CULEX PIPIENS/RESTUANS,41.974689,-87.890615,0,7


In [451]:
# summary of spray data - apparently spray attempts were limited to the below 10 days

query = '''
    select extract(year from date_type) as year,extract(month from date_type) as month,
    count(distinct extract(day from date_type)) from spray
    group by extract(year from date_type),extract(month from date_type)
    order by 1,2
    ;
'''

pd_sql.read_sql(query, connection)

,year,month,count
0,2011.0,8.0,1
1,2011.0,9.0,1
2,2013.0,7.0,2
3,2013.0,8.0,5
4,2013.0,9.0,1


In [364]:
# convert lat,long to radians for calculations within psql

cursor.execute('BEGIN;')
query = '''
    ALTER TABLE spray 
    ADD COLUMN lat_radians DOUBLE PRECISION,
    ADD COLUMN long_radians DOUBLE PRECISION
    ;
    ALTER TABLE virus 
    ADD COLUMN lat_radians DOUBLE PRECISION,
    ADD COLUMN long_radians DOUBLE PRECISION
    ;
    UPDATE spray
    SET lat_radians = RADIANS(latitude),
        long_radians = RADIANS(longitude)
    ;
    UPDATE virus
    SET lat_radians = RADIANS(latitude),
        long_radians = RADIANS(longitude)
    ;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [365]:
# create view for mosquito trap data that includes the distance from the sprayed areas
# using flat earth approximation

cursor.execute('BEGIN;')

# in miles
# round requires (numeric,int) 
query = '''
    CREATE OR REPLACE VIEW spray_all AS
    SELECT DISTINCT t1.date_type date,t2.time,
    t1.latitude, t1.longitude,
    ROUND(CAST(3956* sqrt((pi()/2 - t1.lat_radians)^2 + (pi()/2 - t2.lat_radians)^2 - 2 * 
         (pi()/2 - t1.lat_radians) * (pi()/2 - t2.lat_radians) * 
         cos(t2.long_radians - t1.long_radians)) AS NUMERIC),2) spray_dist
    FROM virus t1 INNER JOIN spray t2
    ON t1.date_type = t2.date_type
    ;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [484]:
# create new view that includes all mosquito trap data with the spray data included from above
# studies have shown that Zenifex (the spray) is most effective within 300 ft = 0.05 mi 
# include flag that indicates whether the area was sprayed based on distance within 0.05 mi

cursor.execute('BEGIN;')

query = '''
    CREATE OR REPLACE VIEW virus_join AS 
    WITH spray_closest AS (
        SELECT EXTRACT(YEAR FROM date) AS year, EXTRACT(MONTH FROM date) AS month,
        latitude,longitude,MIN(spray_dist) AS spray_dist
        FROM spray_all
        WHERE spray_dist is not null
        GROUP BY EXTRACT(MONTH FROM date),EXTRACT(YEAR FROM date),
        latitude,longitude
        ),
    spray_lag AS (
        SELECT year,month,latitude,longitude,spray_dist,
        LAG(spray_dist) OVER(PARTITION BY latitude,longitude ORDER BY year,month) AS last_spray_dist
        FROM spray_closest
        ),
    spray_prev AS (
        SELECT EXTRACT(YEAR FROM TO_DATE(year||'-'||month,'YYYY-MM')+interval '-1 month') AS year,
        EXTRACT(MONTH FROM TO_DATE(year||'-'||month,'YYYY-MM')+interval '-1 month') AS month,
        latitude,longitude,1 AS spray_targeted_prev
        FROM spray_closest
        WHERE spray_dist <= 0.05
        )
    SELECT DISTINCT t1.date_type date,t1.virus_present,
    EXTRACT(YEAR FROM t1.date_type) as year,
    EXTRACT(MONTH FROM t1.date_type) as month,
    t1.species,t1.latitude,t1.longitude,
    SUM(t1.freq) OVER(PARTITION BY t1.date_type,t1.species,t1.block,t1.trap,t1.latitude,t1.longitude) as mosquito_cnt,
    t2.tmax,t2.tmin,t2.tavg,t2.delta,t2.dewpt,t2.wetbulb,
    t2.heat,t2.cool,
    CAST(TRIM(SUBSTR(t2.sunrise,1,2)) AS NUMERIC)+CAST(SUBSTR(t2.sunrise,3,2) AS NUMERIC)/60.00 AS sunrise,
    CAST(TRIM(SUBSTR(t2.sunset,1,2)) AS NUMERIC)+CAST(SUBSTR(t2.sunset,3,2) AS NUMERIC)/60.00 AS sunset,
    t2.codesum,t2.snowdepth,
    t2.snowfall,t2.precip,t2.pressure,t2.sealvlpressure,t2.windspeed,t2.winddirection,t2.avgspeed,
    t3.spray_dist,t3.last_spray_dist,COALESCE(t4.spray_targeted_prev,0) AS spray_targeted_prev
    FROM virus t1 
    LEFT JOIN weather_condensed t2 
    ON t1.date_type = t2.date 
    LEFT JOIN spray_lag t3 
    ON EXTRACT(YEAR FROM t1.date_type) = t3.year AND EXTRACT(MONTH FROM t1.date_type) = t3.month
    AND t1.latitude = t3.latitude AND t1.longitude = t3.longitude
    LEFT JOIN spray_prev t4
    ON EXTRACT(YEAR FROM t1.date_type) = t4.year AND EXTRACT(MONTH FROM t1.date_type) = t4.month
    ;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [483]:
# check duplicates again to see if any remain in downstream view
# block and trap are identifiers that have the same info as latitude and longitude so they have been excluded
pd_sql.read_sql('SELECT date,year,month,species,latitude,longitude,virus_present,count(*) FROM virus_join GROUP BY date,year,month,species,latitude,longitude,virus_present HAVING COUNT(*)>1 ORDER BY 8 DESC;', connection)

,date,year,month,species,latitude,longitude,virus_present,count


In [499]:
# actually, turns out the way the data was collected is actually quite stupid because each "cohort" is not uniquely identifiable
# example from below - for the same location on the same day, there was no virus present in a group of 81 but there was at least one with the virus in a group of 98. 
# but these groups are in the same trap at the same location and you don't know whether the data could have equally been 178 negative and 1 positive
# 2007-08-17	CULEX PIPIENS	41.659112	-87.538693	0	50
# 2007-08-17	CULEX PIPIENS	41.659112	-87.538693	0	31
# 2007-08-17	CULEX PIPIENS	41.659112	-87.538693	1	50
# 2007-08-17	CULEX PIPIENS	41.659112	-87.538693	1	48

query = '''
    SELECT date,species,latitude,longitude,virus_present,freq
    FROM virus 
    WHERE (date,species,latitude,longitude) 
    IN (SELECT date,species,latitude,longitude
        FROM virus
        GROUP BY date,species,latitude,longitude
        HAVING count(*) > 1)
    ORDER BY latitude,longitude,date,species,virus_present
    ;
'''

pd_sql.read_sql(query, connection)

,date,species,latitude,longitude,virus_present,freq
0,2007-08-01,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,0,4
1,2007-08-01,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,0,7
2,2007-08-02,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,0,20
3,2007-08-02,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,0,1
4,2007-08-17,CULEX PIPIENS,41.659112,-87.538693,0,50
5,2007-08-17,CULEX PIPIENS,41.659112,-87.538693,0,31
6,2007-08-17,CULEX PIPIENS,41.659112,-87.538693,1,50
7,2007-08-17,CULEX PIPIENS,41.659112,-87.538693,1,48
8,2007-08-17,CULEX PIPIENS/RESTUANS,41.659112,-87.538693,0,39
9,2007-08-17,CULEX PIPIENS/RESTUANS,41.659112,-87.538693,0,49


In [19]:
# for instances of positive virus, find the mosquito count of the cohort

query = '''
    WITH virus_max as (
    SELECT DISTINCT virus.*,
    MAX(virus_present) OVER(PARTITION BY date_type,species,latitude,longitude) AS virus_max
    FROM virus 
    ORDER BY 3,4,1,2
    )
    SELECT DISTINCT t1.date_type date,t1.virus_present,
    EXTRACT(YEAR FROM t1.date_type) as year,
    EXTRACT(MONTH FROM t1.date_type) as month,
    t1.species,t1.latitude,t1.longitude,
    SUM(t1.freq) OVER(PARTITION BY t1.date_type,t1.species,t1.latitude,t1.longitude) as mosquito_cnt
    FROM virus_max t1
    WHERE t1.virus_present = t1.virus_max
    ORDER BY 6,7,3,4,1,2
    LIMIT 10
    ;
'''

pd_sql.read_sql(query, connection)

,date,virus_present,year,month,species,latitude,longitude,mosquito_cnt
0,2007-08-01,0,2007.0,8.0,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,11
1,2007-08-02,0,2007.0,8.0,CULEX PIPIENS,41.644612,-87.604498,6
2,2007-08-02,0,2007.0,8.0,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,21
3,2007-08-07,0,2007.0,8.0,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,18
4,2007-08-07,0,2007.0,8.0,CULEX PIPIENS,41.644612,-87.604498,26
5,2007-08-15,0,2007.0,8.0,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,10
6,2007-08-15,0,2007.0,8.0,CULEX PIPIENS,41.644612,-87.604498,15
7,2007-08-22,0,2007.0,8.0,CULEX PIPIENS,41.644612,-87.604498,2
8,2007-08-22,1,2007.0,8.0,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,10
9,2007-08-24,0,2007.0,8.0,CULEX PIPIENS/RESTUANS,41.644612,-87.604498,2


In [20]:
# redefine virus_join as a result of above information

cursor.execute('BEGIN;')

query = '''
    CREATE OR REPLACE VIEW virus_join AS 
    WITH spray_closest AS (
        SELECT EXTRACT(YEAR FROM date) AS year, EXTRACT(MONTH FROM date) AS month,
        latitude,longitude,MIN(spray_dist) AS spray_dist
        FROM spray_all
        WHERE spray_dist is not null
        GROUP BY EXTRACT(MONTH FROM date),EXTRACT(YEAR FROM date),
        latitude,longitude
        ),
    spray_lag AS (
        SELECT year,month,latitude,longitude,spray_dist,
        LAG(spray_dist) OVER(PARTITION BY latitude,longitude ORDER BY year,month) AS last_spray_dist
        FROM spray_closest
        ),
    spray_prev AS (
        SELECT EXTRACT(YEAR FROM TO_DATE(year||'-'||month,'YYYY-MM')+interval '-1 month') AS year,
        EXTRACT(MONTH FROM TO_DATE(year||'-'||month,'YYYY-MM')+interval '-1 month') AS month,
        latitude,longitude,1 AS spray_targeted_prev
        FROM spray_closest
        WHERE spray_dist <= 0.05
        ),
    virus_max as (
        SELECT DISTINCT virus.*,
        MAX(virus_present) OVER(PARTITION BY date_type,species,latitude,longitude) AS virus_max
        FROM virus 
        ORDER BY 3,4,1,2
    ),
    virus_fix as (
        SELECT DISTINCT t1.date_type date,t1.virus_present,
        EXTRACT(YEAR FROM t1.date_type) as year,
        EXTRACT(MONTH FROM t1.date_type) as month,
        t1.species,t1.latitude,t1.longitude,
        SUM(t1.freq) OVER(PARTITION BY t1.date_type,t1.species,t1.latitude,t1.longitude) as mosquito_cnt
        FROM virus_max t1
        WHERE t1.virus_present = t1.virus_max
        ORDER BY 6,7,3,4,1,2
    )
    SELECT DISTINCT t1.*,
    t2.tmax,t2.tmin,t2.tavg,t2.delta,t2.dewpt,t2.wetbulb,
    t2.heat,t2.cool,
    CAST(TRIM(SUBSTR(t2.sunrise,1,2)) AS NUMERIC)+CAST(SUBSTR(t2.sunrise,3,2) AS NUMERIC)/60.00 AS sunrise,
    CAST(TRIM(SUBSTR(t2.sunset,1,2)) AS NUMERIC)+CAST(SUBSTR(t2.sunset,3,2) AS NUMERIC)/60.00 AS sunset,
    t2.codesum,t2.snowdepth,
    t2.snowfall,t2.precip,t2.pressure,t2.sealvlpressure,t2.windspeed,t2.winddirection,t2.avgspeed,
    t3.spray_dist,t3.last_spray_dist,COALESCE(t4.spray_targeted_prev,0) AS spray_targeted_prev
    FROM virus_fix t1 
    LEFT JOIN weather_condensed t2 
    ON t1.date = t2.date 
    LEFT JOIN spray_lag t3 
    ON t1.year = t3.year AND t1.month = t3.month
    AND t1.latitude = t3.latitude AND t1.longitude = t3.longitude
    LEFT JOIN spray_prev t4
    ON t1.year = t4.year AND t4.month = t4.month
    ;
'''

cursor.execute(query)
cursor.execute('COMMIT;')

In [21]:
# finally the cleaned/joined data can be pulled down
df_data = pd_sql.read_sql('SELECT * FROM virus_join;', connection)

In [14]:
df_data.columns

Index(['date', 'virus_present', 'year', 'month', 'species', 'latitude',
       'longitude', 'mosquito_cnt', 'tmax', 'tmin', 'tavg', 'delta', 'dewpt',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowdepth',
       'snowfall', 'precip', 'pressure', 'sealvlpressure', 'windspeed',
       'winddirection', 'avgspeed', 'spray_dist', 'last_spray_dist',
       'spray_targeted_prev'],
      dtype='object')

In [22]:
df_data['spray_targeted'] = df_data['spray_dist']<=0.05

In [23]:
df_data.to_pickle('pickles/df_data.pkl')

In [24]:
df_data.head()

,date,virus_present,year,month,species,latitude,longitude,mosquito_cnt,tmax,tmin,...,precip,pressure,sealvlpressure,windspeed,winddirection,avgspeed,spray_dist,last_spray_dist,spray_targeted_prev,spray_targeted
0,2009-06-19,0,2009.0,6.0,CULEX PIPIENS/RESTUANS,41.991429,-87.747113,11,86,65,...,3.97,28.97,29.70,7.2,18.0,11.6,NaN,NaN,0,False
1,2007-08-09,0,2007.0,8.0,CULEX TERRITANS,41.678618,-87.559308,19,84,72,...,0.05,29.13,29.84,2.5,29.0,6.0,NaN,NaN,0,False
2,2009-09-14,0,2009.0,9.0,CULEX PIPIENS/RESTUANS,41.948167,-87.730698,17,84,58,...,0.00,29.34,30.07,0.1,33.0,2.1,NaN,NaN,0,False
3,2007-08-15,0,2007.0,8.0,CULEX SALINARIUS,41.702724,-87.536497,1,80,71,...,0.23,29.17,29.89,3.4,7.0,5.8,NaN,NaN,0,False
4,2013-09-19,0,2013.0,9.0,CULEX PIPIENS/RESTUANS,41.964242,-87.757639,3,84,69,...,0.84,29.14,29.87,8.3,16.0,10.6,NaN,NaN,1,False


In [25]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8475 entries, 0 to 8474
Data columns (total 31 columns):
date                   8475 non-null object
virus_present          8475 non-null int64
year                   8475 non-null float64
month                  8475 non-null float64
species                8475 non-null object
latitude               8475 non-null float64
longitude              8475 non-null float64
mosquito_cnt           8475 non-null int64
tmax                   8475 non-null int64
tmin                   8475 non-null int64
tavg                   8475 non-null float64
delta                  8475 non-null float64
dewpt                  8475 non-null float64
wetbulb                8475 non-null float64
heat                   8475 non-null float64
cool                   8475 non-null float64
sunrise                8475 non-null float64
sunset                 8475 non-null float64
codesum                8475 non-null object
snowdepth              8475 non-null float64
snow